In [44]:
import os

In [45]:
# use GPU 1
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [46]:
IMAGES_DIR = '/d2/caches/kaggle-planet/processed-images-128x128'
IMAGE_SHAPE = (128, 128, 4)
TEST_TIFFS_DIR = '/datasets/kaggle/planet/test-tif-v2'
MODEL = '/d2/caches/kaggle-planet/models/E/e00-l=0.16684-vl=0.14902.h5'
THRESHOLD = 0.25

In [47]:
%run 'Lib.ipynb'

MODEL_CHECKPOINT_DIR not defined 


In [48]:
import keras

In [49]:
X_files_pred = [x.split('.')[0] for x in list_files(TEST_TIFFS_DIR)]
print(len(X_files_pred))

61191


In [50]:
model = keras.models.load_model(MODEL)

In [51]:
subm = []

import re

classes_misc = np.load('out/classes_misc.npy').tolist()
classes_weather = np.load('out/classes_weather.npy').tolist()

i = 0

for f in tqdm(X_files_pred):
    assert (os.path.isfile(IMAGES_DIR + '/' + f))

    b = generate_batch(
        n_samples=1,
        batch_index=i,
        X_files=X_files_pred,
        Y_tags_misc=None,
        Y_tags_weather=None,
        images_dir=IMAGES_DIR,
        image_shape=IMAGE_SHAPE)

    p = model.predict_on_batch(b)

    tags = []

    tags_misc = p[0][0]
    tags_weather = p[1][0]

    # misc
    for j, v in enumerate(tags_misc):
        if v > THRESHOLD:  # cut-off
            tags.append(classes_misc[j])

    # weather
    tag_weather = classes_weather[np.argmax(tags_weather)]
    if 'unknown' != tag_weather: tags.append(tag_weather)

    subm.append({'image_name': f, 'tags': ' '.join(tags)})

    i += 1

100%|██████████| 61191/61191 [03:07<00:00, 326.89it/s]


In [52]:
subm_df = pd.DataFrame(subm)

In [53]:
subm_df.to_csv('out/submission-m.csv', index=False)

In [11]:
# t = [0.07, 0.17, 0.2, 0.04, 0.23, 0.33, 0.24, 0.22, 0.1, 0.19, 0.23, 0.24, 0.12, 0.14, 0.25, 0.26, 0.16]

In [12]:
# np.mean(t[:13])